In [ ]:
import requests
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from typing import List, Dict, Any
import calendar

class EndDateTenorUpdater:
    def __init__(self, api_base_url: str = "http://localhost:3000/api/contracts"):
        self.api_base_url = api_base_url
    
    def fetch_all_contracts(self) -> List[Dict[str, Any]]:
        """Fetch all contracts from the database"""
        try:
            response = requests.get(self.api_base_url)
            if response.status_code == 200:
                contracts = response.json()
                print(f"✓ Fetched {len(contracts)} contracts from database")
                return contracts
            else:
                print(f"✗ Error fetching contracts: {response.status_code}")
                return []
        except Exception as e:
            print(f"✗ Error fetching contracts: {e}")
            return []
    
    def get_month_end_date(self, date_str: str) -> str:
        """Convert a date to the last day of that month"""
        try:
            date = datetime.strptime(date_str, '%Y-%m-%d')
            # Get the last day of the month
            last_day = calendar.monthrange(date.year, date.month)[1]
            month_end = datetime(date.year, date.month, last_day)
            return month_end.strftime('%Y-%m-%d')
        except Exception as e:
            print(f"Error converting date {date_str}: {e}")
            return date_str
    
    def calculate_tenor_from_dates(self, start_date: str, end_date: str) -> Dict[str, Any]:
        """Calculate tenor from start and end dates"""
        try:
            start = datetime.strptime(start_date, '%Y-%m-%d')
            end = datetime.strptime(end_date, '%Y-%m-%d')
            
            # Calculate difference in months
            months_diff = (end.year - start.year) * 12 + (end.month - start.month)
            
            # If the end date is at month end and start is at month start, 
            # and days align well, prefer exact month/year calculation
            years = months_diff // 12
            remaining_months = months_diff % 12
            
            # Determine best representation
            if months_diff % 12 == 0 and months_diff >= 12:
                # Even years
                return {
                    'value': years,
                    'unit': 'years'
                }
            elif months_diff < 24:
                # Less than 2 years, show in months
                return {
                    'value': months_diff,
                    'unit': 'months'
                }
            elif remaining_months == 0:
                # Exact years
                return {
                    'value': years,
                    'unit': 'years'
                }
            else:
                # Mixed, show in months for precision
                return {
                    'value': months_diff,
                    'unit': 'months'
                }
        except Exception as e:
            print(f"Error calculating tenor for {start_date} to {end_date}: {e}")
            return {
                'value': 1,
                'unit': 'years'
            }
    
    def update_contract_dates_and_tenor(self, contract: Dict[str, Any]) -> bool:
        """Update a single contract's end date and tenor"""
        try:
            contract_id = contract.get('_id')
            if not contract_id:
                print(f"✗ No ID found for contract: {contract.get('name', 'Unknown')}")
                return False
            
            contract_name = contract.get('name', 'Unknown')
            start_date = contract.get('startDate')
            end_date = contract.get('endDate')
            
            if not start_date or not end_date:
                print(f"⚠️  Missing dates for contract: {contract_name}")
                return False
            
            # Calculate new end date (month end)
            new_end_date = self.get_month_end_date(end_date)
            
            # Calculate tenor from corrected dates
            tenor = self.calculate_tenor_from_dates(start_date, new_end_date)
            
            # Check if updates are needed
            needs_update = False
            update_payload = {'id': contract_id}
            
            if new_end_date != end_date:
                needs_update = True
                update_payload['endDate'] = new_end_date
                print(f"📅 {contract_name}: End date {end_date} → {new_end_date}")
            
            # Always update tenor if not present or if dates changed
            current_tenor = contract.get('tenor', {})
            if (not current_tenor or 
                current_tenor.get('value') != tenor['value'] or 
                current_tenor.get('unit') != tenor['unit'] or
                needs_update):
                needs_update = True
                update_payload['tenor'] = tenor
                print(f"⏱️  {contract_name}: Tenor → {tenor['value']} {tenor['unit']}")
            
            if not needs_update:
                print(f"✓ {contract_name}: No updates needed")
                return True
            
            # Make API call to update contract
            response = requests.put(self.api_base_url, json=update_payload)
            
            if response.status_code == 200:
                print(f"✅ Updated contract: {contract_name}")
                return True
            else:
                print(f"✗ Error updating contract {contract_name}: {response.status_code} - {response.text}")
                return False
                
        except Exception as e:
            print(f"✗ Error updating contract {contract.get('name', 'Unknown')}: {e}")
            return False
    
    def preview_changes(self, contracts: List[Dict[str, Any]]) -> None:
        """Preview what changes will be made"""
        print("\n" + "="*80)
        print("PREVIEW OF CHANGES")
        print("="*80)
        
        end_date_changes = 0
        tenor_updates = 0
        no_changes = 0
        
        for contract in contracts:
            name = contract.get('name', 'Unknown')
            start_date = contract.get('startDate')
            end_date = contract.get('endDate')
            current_tenor = contract.get('tenor', {})
            
            if not start_date or not end_date:
                print(f"⚠️  {name}: Missing start or end date")
                continue
            
            # Calculate what the new values would be
            new_end_date = self.get_month_end_date(end_date)
            new_tenor = self.calculate_tenor_from_dates(start_date, new_end_date)
            
            changes = []
            
            if new_end_date != end_date:
                changes.append(f"End date: {end_date} → {new_end_date}")
                end_date_changes += 1
            
            if (not current_tenor or 
                current_tenor.get('value') != new_tenor['value'] or 
                current_tenor.get('unit') != new_tenor['unit']):
                old_tenor = f"{current_tenor.get('value', 'N/A')} {current_tenor.get('unit', 'N/A')}" if current_tenor else "None"
                changes.append(f"Tenor: {old_tenor} → {new_tenor['value']} {new_tenor['unit']}")
                tenor_updates += 1
            
            if changes:
                print(f"🔄 {name}:")
                for change in changes:
                    print(f"   {change}")
            else:
                no_changes += 1
        
        print(f"\nSummary:")
        print(f"  Contracts needing end date updates: {end_date_changes}")
        print(f"  Contracts needing tenor updates: {tenor_updates}")
        print(f"  Contracts with no changes needed: {no_changes}")
        print(f"  Total contracts: {len(contracts)}")
    
    def update_all_contracts(self, dry_run: bool = True) -> None:
        """Main method to update all contracts with corrected end dates and tenors"""
        print("Starting database end date and tenor correction process...")
        print(f"Mode: {'DRY RUN (preview only)' if dry_run else 'LIVE UPDATE'}")
        
        # Fetch all contracts
        contracts = self.fetch_all_contracts()
        if not contracts:
            print("No contracts found. Exiting.")
            return
        
        # Preview changes
        self.preview_changes(contracts)
        
        if dry_run:
            print(f"\n{'='*80}")
            print("DRY RUN COMPLETE")
            print("="*80)
            print("This was a preview. To execute the updates, run with dry_run=False")
            return
        
        # Confirm before proceeding with live updates
        print(f"\n{'='*80}")
        print("READY TO EXECUTE LIVE UPDATES")
        print("="*80)
        confirm = input("Are you sure you want to proceed? (y/N): ").strip().lower()
        
        if confirm != 'y':
            print("Update cancelled.")
            return
        
        # Execute updates
        print("\nExecuting updates...")
        success_count = 0
        error_count = 0
        
        for contract in contracts:
            if self.update_contract_dates_and_tenor(contract):
                success_count += 1
            else:
                error_count += 1
        
        # Final summary
        print(f"\n{'='*80}")
        print("UPDATE SUMMARY")
        print("="*80)
        print(f"Successfully updated: {success_count}")
        print(f"Errors: {error_count}")
        print(f"Total processed: {len(contracts)}")
        
        if error_count == 0:
            print("🎉 All contracts updated successfully!")
        else:
            print(f"⚠️  {error_count} contracts had errors during update")

# Example usage and test cases
def test_date_calculations():
    """Test the date calculation functions"""
    updater = EndDateTenorUpdater()
    
    print("Testing date calculations:")
    print("="*50)
    
    test_cases = [
        ("2025-01-01", "2025-12-01"),  # Should become 2025-12-31, 1 year
        ("2025-01-01", "2026-01-01"),  # Should become 2026-01-31, 1 year 1 month  
        ("2025-06-01", "2025-06-01"),  # Should become 2025-06-30, 0 months
        ("2025-01-15", "2027-01-15"),  # Should become 2027-01-31, 2 years
        ("2025-03-01", "2025-09-01"),  # Should become 2025-09-30, 6 months
    ]
    
    for start, end in test_cases:
        new_end = updater.get_month_end_date(end)
        tenor = updater.calculate_tenor_from_dates(start, new_end)
        print(f"Start: {start}, End: {end} → {new_end}, Tenor: {tenor['value']} {tenor['unit']}")

if __name__ == "__main__":
    # Test the calculations first
    test_date_calculations()
    
    print("\n" + "="*80)
    print("STARTING CONTRACT DATABASE UPDATE")
    print("="*80)
    
    updater = EndDateTenorUpdater("http://localhost:3000/api/contracts")
    
    # First run a dry run to preview changes
    print("=== DRY RUN: Preview Changes ===")
    updater.update_all_contracts(dry_run=False)
    
    # Uncomment the line below to execute the actual updates
    # print("\n=== LIVE UPDATE: Execute Changes ===")
    # updater.update_all_contracts(dry_run=False)

Testing date calculations:
Start: 2025-01-01, End: 2025-12-01 → 2025-12-31, Tenor: 11 months
Start: 2025-01-01, End: 2026-01-01 → 2026-01-31, Tenor: 1 years
Start: 2025-06-01, End: 2025-06-01 → 2025-06-30, Tenor: 0 months
Start: 2025-01-15, End: 2027-01-15 → 2027-01-31, Tenor: 2 years
Start: 2025-03-01, End: 2025-09-01 → 2025-09-30, Tenor: 6 months

STARTING CONTRACT DATABASE UPDATE
=== DRY RUN: Preview Changes ===
Starting database end date and tenor correction process...
Mode: DRY RUN (preview only)
✓ Fetched 354 contracts from database

PREVIEW OF CHANGES
🔄 BVC TOU:
   End date: 2025-06-01 → 2025-06-30
   Tenor: None → 3 months
🔄 BVC TOU (LGC):
   End date: 2025-06-01 → 2025-06-30
   Tenor: None → 3 months
🔄 CSIRO TOU:
   End date: 2030-06-01 → 2030-06-30
   Tenor: None → 63 months
🔄 CSIRO TOU (LGC):
   End date: 2030-06-01 → 2030-06-30
   Tenor: None → 63 months
🔄 Fairfield Non-TOU:
   End date: 2026-12-01 → 2026-12-31
   Tenor: None → 21 months
🔄 Fairfield Non-TOU (LGC):
   End da